# GPUs
:label:`sec_use_gpu`

Em :numref:`tab_intro_decade`, discutimos o rápido crescimento
de computação nas últimas duas décadas.
Em suma, o desempenho da GPU aumentou
por um fator de 1000 a cada década desde 2000.
Isso oferece ótimas oportunidades, mas também sugere
uma necessidade significativa de fornecer tal desempenho.

Nesta seção, começamos a discutir como aproveitar
este desempenho computacional para sua pesquisa.
Primeiro usando GPUs únicas e, posteriormente,
como usar várias GPUs e vários servidores (com várias GPUs).

Especificamente, discutiremos como
para usar uma única GPU NVIDIA para cálculos.
Primeiro, certifique-se de ter pelo menos uma GPU NVIDIA instalada.
Em seguida, baixe o [NVIDIA driver e CUDA](https://developer.nvidia.com/cuda-downloads).
e siga as instruções para definir o caminho apropriado.
Assim que esses preparativos forem concluídos,
o comando `nvidia-smi` pode ser usado
para ver as informações da placa gráfica.


In [1]:
!nvidia-smi

Sat Dec 11 07:57:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   40C    P0    50W / 300W |     11MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+


|   1  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   36C    P0    50W / 300W |    584MiB / 16130MiB |      5%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2...  Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   73C    P0   276W / 300W |   8892MiB / 16130MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   3  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   51C    P0    48W / 300W |     11MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name  

Para executar os programas desta seção,
você precisa de pelo menos duas GPUs.
Observe que isso pode ser extravagante para a maioria dos computadores desktop
mas está facilmente disponível na nuvem, por exemplo,
usando as instâncias multi-GPU do AWS EC2.
Quase todas as outras seções * não * requerem várias GPUs.
Em vez disso, isso é simplesmente para ilustrar
como os dados fluem entre diferentes dispositivos.

## Dispositivos Computacionais

Podemos especificar dispositivos, como CPUs e GPUs,
para armazenamento e cálculo.
Por padrão, os tensores são criados na memória principal
e, em seguida, use a CPU para calculá-lo.


In [2]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7ff644556550>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7ff6445566d0>)

Podemos consultar o número de GPUs disponíveis.


In [3]:
len(tf.config.experimental.list_physical_devices('GPU'))

2

Agora definimos duas funções convenientes que nos permitem
para executar o código mesmo que as GPUs solicitadas não existam.


In [4]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7ff59d9e6b20>,
  <tensorflow.python.eager.context._EagerDeviceContext at 0x7ff59004d1c0>])

## Tensores e GPUs

Por padrão, tensores são criados na CPU.
Podemos consultar o dispositivo onde o tensor está localizado.


In [5]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

É importante notar que sempre que quisermos
para operar em vários termos,
eles precisam estar no mesmo dispositivo.
Por exemplo, se somarmos dois tensores,
precisamos ter certeza de que ambos os argumentos
estão no mesmo dispositivo --- caso contrário, a estrutura
não saberia onde armazenar o resultado
ou mesmo como decidir onde realizar o cálculo.

### Armazenamento na GPU

Existem várias maneiras de armazenar um tensor na GPU.
Por exemplo, podemos especificar um dispositivo de armazenamento ao criar um tensor.
A seguir, criamos a variável tensorial `X` no primeiro `gpu`.
O tensor criado em uma GPU consome apenas a memória desta GPU.
Podemos usar o comando `nvidia-smi` para ver o uso de memória da GPU.
Em geral, precisamos ter certeza de não criar dados que excedam o limite de memória da GPU.


In [6]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

Supondo que você tenha pelo menos duas GPUs, o código a seguir criará um tensor aleatório na segunda GPU.


In [7]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.9953705 , 0.2957828 , 0.80567956],
       [0.11053193, 0.7893543 , 0.5817497 ]], dtype=float32)>

### Copiando

Se quisermos calcular `X + Y`,
precisamos decidir onde realizar esta operação.
Por exemplo, como mostrado em :numref:`fig_copyto`,
podemos transferir `X` para a segunda GPU
e realizar a operação lá.
*Não* simplesmente adicione `X` e` Y`,
pois isso resultará em uma exceção.
O mecanismo de tempo de execução não saberia o que fazer:
ele não consegue encontrar dados no mesmo dispositivo e falha.
Já que `Y` vive na segunda GPU,
precisamos mover `X` para lá antes de podermos adicionar os dois.

![Copiar dados para realizar uma operação no mesmo dispositivo.](http://d2l.ai/_images/copyto.svg)
:label:`fig_copyto`


In [8]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


Agora que os dados estão na mesma GPU
(ambos são `Z` e` Y`),
podemos somá-los.


In [9]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.9953705, 1.2957828, 1.8056796],
       [1.1105319, 1.7893543, 1.5817497]], dtype=float32)>

Imagine que sua variável `Z` já esteja em sua segunda GPU.
O que acontece se ainda chamarmos `Z2 = Z` no mesmo escopo de dispositivo?
Ele retornará `Z` em vez de fazer uma cópia e alocar nova memória.


In [10]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

### Informações extra

As pessoas usam GPUs para fazer aprendizado de máquina
porque eles esperam que ela seja rápida.
Mas a transferência de variáveis entre dispositivos é lenta.
Então, queremos que você tenha 100% de certeza
que você deseja fazer algo lento antes de deixá-lo fazer.
Se a estrutura de *Deep Learning* apenas fizesse a cópia automaticamente
sem bater, então você pode não perceber
que você escreveu algum código lento.

Além disso, a transferência de dados entre dispositivos (CPU, GPUs e outras máquinas)
é algo muito mais lento do que a computação.
Também torna a paralelização muito mais difícil,
já que temos que esperar que os dados sejam enviados (ou melhor, para serem recebidos)
antes de prosseguirmos com mais operações.
É por isso que as operações de cópia devem ser realizadas com muito cuidado.
Como regra geral, muitas pequenas operações
são muito piores do que uma grande operação.
Além disso, várias operações ao mesmo tempo
são muito melhores do que muitas operações simples intercaladas no código
a menos que você saiba o que está fazendo.
Este é o caso, uma vez que tais operações podem bloquear se um dispositivo
tem que esperar pelo outro antes de fazer outra coisa.
É um pouco como pedir seu café em uma fila
em vez de pré-encomendá-lo por telefone
e descobrir que ele está pronto quando você estiver.

Por último, quando imprimimos tensores ou convertemos tensores para o formato NumPy,
se os dados não estiverem na memória principal,
o framework irá copiá-lo para a memória principal primeiro,
resultando em sobrecarga de transmissão adicional.
Pior ainda, agora está sujeito ao temido bloqueio de intérprete global
isso faz tudo esperar que o Python seja concluído.


## Redes Neurais e GPUs

Da mesma forma, um modelo de rede neural pode especificar dispositivos.
O código a seguir coloca os parâmetros do modelo na GPU.


In [11]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Veremos muitos mais exemplos de
como executar modelos em GPUs nos capítulos seguintes,
simplesmente porque eles se tornarão um pouco mais intensivos em termos de computação.

Quando a entrada é um tensor na GPU, o modelo calculará o resultado na mesma GPU.


In [12]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.97097045],
       [-0.97097045]], dtype=float32)>

Vamos confirmar se os parâmetros do modelo estão armazenados na mesma GPU.


In [13]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')

Resumindo, contanto que todos os dados e parâmetros estejam no mesmo dispositivo, podemos aprender modelos com eficiência. Nos próximos capítulos, veremos vários desses exemplos.

## Sumário

* Podemos especificar dispositivos para armazenamento e cálculo, como CPU ou GPU.
   Por padrão, os dados são criados na memória principal
   e então usa-se a CPU para cálculos.
* A estrutura de *Deep Learning* requer todos os dados de entrada para cálculo
   estar no mesmo dispositivo,
   seja CPU ou a mesma GPU.
* Você pode perder um desempenho significativo movendo dados sem cuidado.
   Um erro típico é o seguinte: calcular a perda
   para cada minibatch na GPU e relatando de volta
   para o usuário na linha de comando (ou registrando-o em um NumPy `ndarray`)
   irá disparar um bloqueio global do interpretador que paralisa todas as GPUs.
   É muito melhor alocar memória
   para registrar dentro da GPU e apenas mover registros maiores.

## Exercícios

1. Tente uma tarefa de computação maior, como a multiplicação de grandes matrizes,
    e veja a diferença de velocidade entre a CPU e a GPU.
    Que tal uma tarefa com uma pequena quantidade de cálculos?
1. Como devemos ler e escrever os parâmetros do modelo na GPU?
1. Meça o tempo que leva para calcular 1000
    multiplicações matriz-matriz de $100 \times 100$ matrizes
    e registrar a norma de Frobenius da matriz de saída, um resultado de cada vez
    vs. manter um registro na GPU e transferir apenas o resultado final.
1. Meça quanto tempo leva para realizar duas multiplicações matriz-matriz
    em duas GPUs ao mesmo tempo vs. em sequência
    em uma GPU. Dica: você deve ver uma escala quase linear.


[Discussão](https://discuss.d2l.ai/t/270)


<!--stackedit_data:
eyJoaXN0b3J5IjpbMTM1NzUzMTk4OCwtMTA5NzI3MTAzNiw4MD
c2MDQzNTksLTY3MjIyODU4NF19
-->
